In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.Session()

data_size =[10,10]
data_2d = np.random.normal(size=data_size)
x_input_2d = tf.placeholder(dtype=tf.float32, shape=data_size)

In [3]:
def conv_layer_2d(input_2d, my_fliter):
    #change 2d input to 4d
    input_3d = tf.expand_dims(input_2d, 0)
    print(input_3d)
    input_4d = tf.expand_dims(input_3d, 3)
    print(input_4d)

    #  input_1: an input tensor of shape [batch, in_height, in_width, in_channels]
    #  fliter_2:  a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    # stride_3: Must have strides[0] = strides[3] = 1. For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1].
    convolution_output = tf.nn.conv2d(input_4d, filter=my_fliter, strides=[1,2,2,1], padding="VALID")
    # drop extra dimensions
    conv_output_2d = tf.squeeze(convolution_output)
    print(conv_output_2d)
    return (conv_output_2d)


my_fliter = tf.Variable(tf.random_normal(shape=[2,2,1,1]))
my_convolution_output = conv_layer_2d(x_input_2d, my_fliter)

Tensor("ExpandDims:0", shape=(1, 10, 10), dtype=float32)
Tensor("ExpandDims_1:0", shape=(1, 10, 10, 1), dtype=float32)
Tensor("Squeeze:0", shape=(5, 5), dtype=float32)


In [4]:
def activation(input_2d):
    relurlt =tf.nn.relu(input_2d)
    print(relurlt)
    return relurlt

my_activation_output = activation(my_convolution_output)

Tensor("Relu:0", shape=(5, 5), dtype=float32)


In [5]:
def max_pool(input_2d, width,height):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    print(input_3d)
    print(input_4d)
    
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, height, width, 1], strides=[1,1,1,1], padding="VALID")
    print(pool_output)
    pool_output_2d = tf.squeeze(pool_output)
    print(pool_output_2d)
    return(pool_output_2d)

my_max_output = max_pool(my_activation_output, width=2, height=2)

Tensor("ExpandDims_2:0", shape=(1, 5, 5), dtype=float32)
Tensor("ExpandDims_3:0", shape=(1, 5, 5, 1), dtype=float32)
Tensor("MaxPool:0", shape=(1, 4, 4, 1), dtype=float32)
Tensor("Squeeze_1:0", shape=(4, 4), dtype=float32)


In [6]:
def fully_connected(input_layer, num_outputs):
    flatten_input = tf.reshape(input_layer, [-1])
    print(flatten_input)
    weight_shape= tf.squeeze(tf.stack([tf.shape(flatten_input), [num_outputs]]))
    weight = tf.random_normal(weight_shape, stddev=0.1)
    bias = tf.random_normal(shape=[num_outputs])
    
    input_2d = tf.expand_dims(flatten_input, 0)
    full_output = tf.add(tf.matmul(input_2d, weight), bias)
    
    full_output_2d = tf.squeeze(full_output)
    return(full_output_2d)

my_full_output = fully_connected(my_max_output, 5)

Tensor("Reshape:0", shape=(16,), dtype=float32)


In [7]:
init = tf.global_variables_initializer()
sess.run(init)

feed_dict = {x_input_2d: data_2d}

In [8]:
# Convolution Output 
print('Input = [10 X 10] array') 
print('2x2 Convolution, stride size = [2x2], results in the [5x5] array:') 
print(sess.run(my_convolution_output, feed_dict=feed_dict)) 
# Activation Output 
print('Input = the above [5x5] array') 
print('ReLU element wise returns the [5x5] array:') 
print(sess.run(my_activation_output, feed_dict=feed_dict)) 
# Max Pool Output 
print('Input = the above [5x5] array') 
print('MaxPool, stride size = [1x1], results in the [4x4] array:') 
print(sess.run(my_max_output, feed_dict=feed_dict)) 
# Fully Connected Output 
print('Input = the above [4x4] array') 
print('Fully connected layer on all four rows with five outputs:') 
print(sess.run(my_full_output, feed_dict=feed_dict)) 

Input = [10 X 10] array
2x2 Convolution, stride size = [2x2], results in the [5x5] array:
[[ 4.5575819  -0.6830005  -0.25743484  1.35964024 -2.23150539]
 [ 4.96601439 -0.20229781 -1.8142823  -3.97244406  0.72050661]
 [ 4.40277004  0.91883886  2.37902641 -2.75206614 -0.28397948]
 [-1.69307649  0.35923988  1.1325928  -0.96266013 -1.08061743]
 [-1.25605798 -2.90235901 -2.69128227  0.30081913  1.25056469]]
Input = the above [5x5] array
ReLU element wise returns the [5x5] array:
[[ 4.5575819   0.          0.          1.35964024  0.        ]
 [ 4.96601439  0.          0.          0.          0.72050661]
 [ 4.40277004  0.91883886  2.37902641  0.          0.        ]
 [ 0.          0.35923988  1.1325928   0.          0.        ]
 [ 0.          0.          0.          0.30081913  1.25056469]]
Input = the above [5x5] array
MaxPool, stride size = [1x1], results in the [4x4] array:
[[ 4.96601439  0.          1.35964024  1.35964024]
 [ 4.96601439  2.37902641  2.37902641  0.72050661]
 [ 4.40277004  